In [235]:
import pandas as pd

# Read in CSV File
season_box = pd.read_csv('season_box_scores_20182019.csv', converters={'GAME_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)
season_games = pd.read_csv(
    'season_games_20182019.csv', converters={'Game_ID': str, 'TEAM_ABBREVIATION': str}, index_col=False)\

# Define columns to keep from games dataframe
col_keep = ['W_PCT', 'FG_PCT', 'FG3_PCT',
            'FT_PCT', 'OREB', 'AST', 'STL', 'BLK', 'TOV']


In [236]:
# Group by GAME_ID in season_20_21_box and keep only first two rows
season_box = season_box.groupby(
    'GAME_ID').head(2).reset_index(drop=True)

In [237]:
season_box.head()


,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PTS_PAINT,PTS_2ND_CHANCE,PTS_FB,LARGEST_LEAD,LEAD_CHANGES,TIMES_TIED,...,PTS_OT2,PTS_OT3,PTS_OT4,PTS_OT5,PTS_OT6,PTS_OT7,PTS_OT8,PTS_OT9,PTS_OT10,PTS
0,0,1610612745,HOU,Houston,24,13,12,3,7,4,...,0,0,0,0,0,0,0,0,0,95
1,0,1610612737,ATL,Atlanta,70,23,11,31,7,4,...,0,0,0,0,0,0,0,0,0,124
2,0,1610612753,ORL,Orlando,38,21,8,3,1,0,...,0,0,0,0,0,0,0,0,0,116
3,0,1610612737,ATL,Atlanta,42,10,12,30,1,0,...,0,0,0,0,0,0,0,0,0,93
4,0,1610612764,WAS,Washington,58,15,13,13,3,7,...,0,0,0,0,0,0,0,0,0,120


In [238]:
# Get unique values of Game_ID column in season_20_21_games
game_ids = season_games['Game_ID'].unique()

season_box_ext = pd.DataFrame(columns=season_box.columns)

for id in game_ids:
    # Get all rows with id in Game_ID column
    df_game = season_games[season_games['Game_ID'] == id]
    # Split MATCHUP column string at 'vs.' and strip whitespace
    matchup_split = df_game.iloc[0]['MATCHUP'].split(' ')
    team_one = matchup_split[0].strip()
    team_two = matchup_split[2].strip()
    # Drop columns not in col_keep
    df_game = df_game[col_keep]
    # Make new dataframe out of both rows
    df_team_one = df_game.iloc[0:1]
    df_team_two = df_game.iloc[1:2]
    # Get row matching TEAM_ABBREVIATION and GAME_ID from season_box_clean
    match_row_one = season_box[(season_box['TEAM_ABBREVIATION'] == team_one) & (
        season_box['GAME_ID'] == id)]
    match_row_two = season_box[(season_box['TEAM_ABBREVIATION'] == team_two) & (
        season_box['GAME_ID'] == id)]
    # Concat match_row with df_teams and reset index
    match_row_one.reset_index(drop=True, inplace=True)
    match_row_two.reset_index(drop=True, inplace=True)
    df_team_one.reset_index(drop=True, inplace=True)
    df_team_two.reset_index(drop=True, inplace=True)
    df_match_row_one = pd.concat([match_row_one, df_team_one], axis=1)
    df_match_row_two = pd.concat([match_row_two, df_team_two], axis=1)
    # Concat to season_box_ext
    season_box_ext = pd.concat([season_box_ext, df_match_row_one, df_match_row_two])


In [239]:
season_box_ext.head()
# write season_box_ext to csv
season_box_ext = season_box_ext.to_csv('season_box_ext.csv', index=False)